In [1]:
!wget http://www.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
!unzip shakespeare.zip

URL transformed to HTTPS due to an HSTS policy
--2023-10-11 10:15:51--  https://www.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
Resolving www.lawrence.edu (www.lawrence.edu)... 143.44.124.14
Connecting to www.lawrence.edu (www.lawrence.edu)|143.44.124.14|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www7.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip [following]
--2023-10-11 10:15:51--  https://www7.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
Resolving www7.lawrence.edu (www7.lawrence.edu)... 143.44.124.14
Connecting to www7.lawrence.edu (www7.lawrence.edu)|143.44.124.14|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www2.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip [following]
--2023-10-11 10:15:51--  https://www2.lawrence.edu/fast/greggj/CMSC490/shakespeare.zip
Resolving www2.lawrence.edu (www2.lawrence.edu)... 143.44.124.14
Connecting to www2.lawrence.edu (www2.lawrence.edu)|

**Function that reads single word from file**

In [1]:
def getWord(file):
    char = file.read(1)
    if not char:
        return ''
    char = char.lower()

    while char < 'a' or char > 'z':
        char = file.read(1)        
        if not char: 
            return ''
        char = char.lower()

    str = ''
    while char >= 'a' and char <= 'z':
        str = str + char
        char = file.read(1)        
        if not char: 
            return str
        char = char.lower()
    
    if char == '’':
        str = ''
        char = file.read(1)
        if not char: 
            return str
        char = char.lower()
        while char >= 'a' and char <= 'z':
            char = file.read(1)        
            if not char: 
                return str
            char = char.lower()
        return getWord(file)
    return str

In [2]:
import numpy as np
words = np.zeros(100000, dtype=object)

In [3]:
file = open('shakespeare.txt', 'r')

for n in range(0,len(words)):
    words[n] = getWord(file)
 
file.close()

In [4]:
def makeSequence(ch):
    if ch == ' ':
        position = 26
    else:
        position = ord(ch) - ord('a')
    encodedNum = np.zeros(27)
    encodedNum[position] = 1
    return encodedNum
    
    

In [5]:
substrings = np.zeros((100000,10,27))
targets = np.zeros((100000,27))

In [6]:
def genSubstrings(encoded):
    substring = encoded[:]
    numBlank =10 - len(encoded)
    for i in range(numBlank):
        blank = np.zeros(27)
        blank[26] = 1
        substring.append(blank)
    return substring

In [7]:
k = 0 #to keep track of how many things i've added to the substrings, target array
for i in range(len(words)):
    
    if len(words[i]) < 4 or len(words[i]) > 9:
        continue

    if k >= 100000:
        break
    chars = list(words[i]) # "h", "o", "u", "s", "e"
    encoded = [makeSequence(chars[0]),makeSequence(chars[1]),makeSequence(chars[2])]
    substrings[k] = genSubstrings(encoded)

    targets[k] = makeSequence(chars[3])
    k+=1

    for n in range(3,len(chars)):
        encoded.append(makeSequence(chars[n]))
        substrings[k] = genSubstrings(encoded)
        if n+1 == len(chars): #when the substring is equal to the original string
            targets[k] = makeSequence(' ')
        else:
            targets[k] = makeSequence(chars[n+1])
        k+=1
    
    

In [18]:
X_train = substrings[:60000]
x_test = substrings[60000:80000]
x_val = substrings[80000:]

Y_train = targets[:60000]
y_test = targets[60000:80000]
y_val = targets[80000:]

In [15]:
# import tensorflow as tf
# dataset = tf.data.Dataset.from_tensor_slices((substrings, targets))
# train_size = int(0.6 * len(dataset))
# val_size = int(0.2 * len(dataset))
# test_size = len(dataset) - train_size - val_size

# train_dataset = dataset.take(train_size)
# remaining_dataset = dataset.skip(train_size)
# val_dataset = remaining_dataset.take(val_size)
# test_dataset = remaining_dataset.skip(val_size)

# print(len(dataset))
# print(len(train_dataset))
# print(len(val_dataset))
# print(len(test_dataset))

100000
60000
20000
20000


In [23]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    layers.LSTM(16,input_shape=(10,27),return_sequences=False),
    layers.Dense(27, activation="softmax")
])

In [24]:
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [25]:
history = model.fit(X_train,
                    Y_train,
                    epochs=20,
                    validation_data=(x_val, y_val))

Epoch 1/20
1875/1875 [==============================] - 10s 5ms/step - loss: 2.4062 - accuracy: 0.3484 - val_loss: 2.4091 - val_accuracy: 0.3460
Epoch 2/20
1875/1875 [==============================] - 10s 5ms/step - loss: 2.2287 - accuracy: 0.3850 - val_loss: 2.3232 - val_accuracy: 0.3738
Epoch 3/20
1875/1875 [==============================] - 10s 5ms/step - loss: 2.0980 - accuracy: 0.4188 - val_loss: 2.2163 - val_accuracy: 0.3857
Epoch 4/20
1875/1875 [==============================] - 10s 5ms/step - loss: 1.9784 - accuracy: 0.4561 - val_loss: 2.1162 - val_accuracy: 0.4188
Epoch 5/20
1875/1875 [==============================] - 9s 5ms/step - loss: 1.8815 - accuracy: 0.4823 - val_loss: 2.0582 - val_accuracy: 0.4321
Epoch 6/20
1875/1875 [==============================] - 9s 5ms/step - loss: 1.8041 - accuracy: 0.5035 - val_loss: 2.0085 - val_accuracy: 0.4465
Epoch 7/20
1875/1875 [==============================] - 10s 5ms/step - loss: 1.7473 - accuracy: 0.5186 - val_loss: 1.9801 - val_accu